In [1]:
filename = 'points_NSXm_TMD_40_32_28.json'
In = 28

import numpy as np
import json

def get_relay_curve(In=1, order=4, C=1):
    def relay_curve(I, *ks):
        t = 0
        for order, k in enumerate(ks):
            t += k*np.log10((I/In)-C)**order
        return 10**t
    return relay_curve

with open(filename) as f:
    data = json.load(f)['Points']

x = np.array([row['X'] for row in data])/In
y = np.array([row['Y'] for row in data])
points_n = np.argwhere(x == np.min(x))[0,0]
i_l = x[3:points_n][::-1]
t_l = y[3:points_n][::-1]
i_u = x[points_n+3:]
t_u = y[points_n+3:]

l_filter = t_l <= 3600*2
u_filter = t_u <= 3600*2
i_l = i_l[l_filter]
t_l = t_l[l_filter]
i_u = i_u[u_filter]
t_u = t_u[u_filter]

def reduce(i, t):
    i1 = i[0]
    i2 = i[-1]
    i_n = np.geomspace(i1,i2,num=50)
    i_n_nd = np.tile(i_n, (len(i), 1))
    i_nd = np.tile(i, (len(i_n), 1)).T
    idx = (np.abs(i_n_nd - i_nd)).argmin(axis=0)
    return i[idx], t[idx]

i_u, t_u = reduce(i_u, t_u)
i_l, t_l = reduce(i_l, t_l)

In [2]:
import matplotlib.pyplot as plt
%matplotlib auto

plt.plot(i_u, t_u, 'b-', label='upper')
plt.plot(i_l, t_l, 'r-', label='lower')
plt.xscale('log')
plt.yscale('log')
plt.xlim(1,100)
plt.ylim(0.1,1e4)
plt.grid(True, which='major', alpha=1)
plt.grid(True, which='minor', alpha=0.5)
plt.legend()

Using matplotlib backend: TkAgg


In [3]:
# Curve fitting upper

In = 1.28
order=7
relay_curve = get_relay_curve(In, order)

xdata = np.array(i_u)
ydata = np.array(t_u)

xdata_mod = np.log10((xdata/In) -1)
ydata_mod = np.log10(ydata)
coeff = np.polyfit(xdata_mod, ydata_mod, order)[::-1]

plt.plot(xdata, ydata, 'b-', label='data')
x = np.geomspace(min(xdata), 45, num=100)
plt.plot(x, relay_curve(x, *list(coeff)), 'r-', label='fit')
plt.xscale('log')
plt.yscale('log')
plt.grid(True, which='major', alpha=1)
plt.grid(True, which='minor', alpha=0.5)
plt.legend()
display(list(np.round(coeff,5)))

[2.17722, -2.01847, -0.34372, 0.92348, 0.24174, -0.44728, -0.07033, 0.06749]

In [4]:
# Curve fitting lower

In=1.03
order=7
relay_curve = get_relay_curve(In, order)

xdata = np.array(i_l)
ydata = np.array(t_l)

xdata_mod = np.log10((xdata/In) -1)
ydata_mod = np.log10(ydata)
coeff = np.polyfit(xdata_mod, ydata_mod, order)[::-1]

plt.plot(xdata, ydata, 'b-', label='data')
x = np.geomspace(min(xdata), 30, num=100)
plt.plot(x, relay_curve(x, *list(coeff)), 'r-', label='fit')
plt.xscale('log')
plt.yscale('log')
plt.grid(True, which='major', alpha=1)
plt.grid(True, which='minor', alpha=0.5)
plt.legend()
display(list(np.round(coeff,5)))

[1.31295, -1.88771, -0.08583, 0.66854, 0.23135, -0.22075, -0.10873, -0.00715]

In [5]:
print(np.min(i_l), np.min(i_u))
print(json.dumps(list(np.round(i_u, 5))), '\n')
print(json.dumps(list(np.round(t_u, 5))), '\n')
print(json.dumps(list(np.round(i_l, 5))), '\n')
print(json.dumps(list(np.round(t_l, 5))), '\n')

1.0335714285714286 1.3014285714285752
[1.30143, 1.38357, 1.46929, 1.56214, 1.65857, 1.76571, 1.87643, 1.99429, 2.11929, 2.25143, 2.39071, 2.54071, 2.70143, 2.86929, 3.05143, 3.24429, 3.44786, 3.68, 3.89429, 4.14429, 4.39429, 4.68, 4.96571, 5.28714, 5.60857, 5.96571, 6.32286, 6.71571, 7.14429, 7.60857, 8.07286, 8.57286, 9.10857, 9.71571, 10.32286, 10.96571, 11.64429, 12.35857, 13.14429, 13.96571, 14.85857, 15.78714, 16.78714, 17.82286, 18.93, 20.14429, 21.39429, 22.75143, 24.18, 25.68] 

[6984.38825, 3582.55596, 2287.94159, 1547.48483, 1077.96881, 794.62522, 590.87056, 439.14637, 333.77243, 260.19327, 203.21775, 158.17964, 122.87879, 99.06768, 79.61004, 64.03347, 51.58293, 42.33073, 35.7067, 29.6143, 24.83212, 21.13379, 18.24726, 15.6204, 13.49497, 11.83724, 10.55837, 9.55284, 8.75965, 8.01994, 7.38119, 6.78511, 6.23262, 5.6938, 5.23016, 4.80564, 4.41787, 4.06429, 3.728, 3.42442, 3.13961, 2.88394, 2.64613, 2.43321, 2.23617, 2.0496, 1.8838, 1.72825, 1.58689, 1.45855] 

[1.03357, 1.09429,